<a href="https://colab.research.google.com/github/jiyoungemma/User-Log-data-analytics/blob/main/recommend_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 라이브러리 & 데이터 불러오기

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from datetime import datetime
%matplotlib inline
import pandas as pd
import datetime
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

In [29]:
RFM_without_view = pd.read_csv("/content/drive/MyDrive/RFM_without_view.csv")

In [30]:
RFM_without_view.drop(columns=['field1', 'event_time','user_id.1','category_id', 'Recency','Monetary','Recency_score', 'Frequency_score', 'Monetary_score',
       'RFM_seg', 'RFM_score'], inplace=True)

In [5]:
RFM_without_view.head()

,event_type,category_code,brand,price,user_id,user_session,main_category,subcategory,Frequency
0,purchase,electronics.smartphone,samsung,130.76,543272936,8187d148-3c41-46d4-b0c0-9c08cd9dc564,electronics,smartphone,33
1,purchase,electronics.smartphone,apple,642.69,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68,electronics,smartphone,9
2,purchase,NaN,xiaomi,29.51,514591159,0e5dfc4b-2a55-43e6-8c05-97e1f07fbb56,NaN,NaN,1
3,purchase,furniture.bathroom.toilet,santeri,54.42,555332717,1dea3ee2-2ded-42e8-8e7a-4e2ad6ae942f,furniture,toilet,4
4,purchase,electronics.audio.headphone,apple,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553,electronics,headphone,1


In [77]:
RFM_without_view.loc[RFM_without_view['Frequency'] == 1]

,event_type,product_id,category_code,brand,price,user_id,user_session,main_category,subcategory,Frequency,RFM_label
2,purchase,5100816,NaN,xiaomi,29.51,514591159,0e5dfc4b-2a55-43e6-8c05-97e1f07fbb56,NaN,NaN,1,segment5
4,purchase,4804055,electronics.audio.headphone,apple,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553,electronics,headphone,1,segment5
8,purchase,6200687,appliances.environment.air_heater,oasis,28.03,548691404,b67cdbcb-b073-4271-b365-803c6fce53b0,appliances,air_heater,1,segment5
12,cart,1004833,electronics.smartphone,samsung,174.76,536415846,685b5b42-f597-4a69-ab4c-ef96a30bc454,electronics,smartphone,1,segment5
20,purchase,4804295,electronics.audio.headphone,xiaomi,23.13,540557704,0c990471-9407-4292-a6df-0cbc2f722a40,electronics,headphone,1,segment5
...,...,...,...,...,...,...,...,...,...,...,...
1382811,cart,5100816,NaN,xiaomi,29.58,514905226,6f25d2de-bb1c-4e68-b848-f03372fb6e7d,NaN,NaN,1,segment4
1382814,cart,1004250,electronics.smartphone,apple,787.50,514503441,d2169b3a-422b-4887-9149-ddf35e68e537,electronics,smartphone,1,segment2
1382816,purchase,1004250,electronics.smartphone,apple,787.50,514503441,d2169b3a-422b-4887-9149-ddf35e68e537,electronics,smartphone,1,segment2
1382818,purchase,5100816,NaN,xiaomi,29.58,514905226,6f25d2de-bb1c-4e68-b848-f03372fb6e7d,NaN,NaN,1,segment4


In [78]:
RFM_without_view.shape

(1382831, 11)

In [80]:
(530997 / 1382831) * 100

38.39926932502959

### computers dateframe

In [5]:
com_once = RFM_without_view.loc[(RFM_without_view['Frequency'] < 2) & (RFM_without_view['event_type'] == 'purchase') & (RFM_without_view['main_category'] == 'computers')]

In [6]:
com_once_user = RFM_without_view.loc[(RFM_without_view['Frequency'] < 2) & (RFM_without_view['event_type'] == 'purchase') & (RFM_without_view['main_category'] == 'computers')]

In [7]:
com_once.shape

(9353, 9)

In [8]:
com_once.columns

Index(['event_type', 'category_code', 'brand', 'price', 'user_id',
       'user_session', 'main_category', 'subcategory', 'Frequency'],
      dtype='object')

In [9]:
com_once_user.head()

,event_type,category_code,brand,price,user_id,user_session,main_category,subcategory,Frequency
38,purchase,computers.peripherals.printer,epson,116.73,513489079,63de1120-9d48-45d0-b024-500d1b95d71f,computers,printer,1
68,purchase,computers.components.hdd,kingston,66.67,523117953,629a0485-36c9-4a80-9f02-4a96382c3059,computers,hdd,1
167,purchase,computers.notebook,acer,532.81,549790258,56272bb8-9f35-4412-9f15-624c8ecda0f1,computers,notebook,1
412,purchase,computers.notebook,hp,285.51,555467053,8d2582ff-8031-4b06-819b-a653eb1cb832,computers,notebook,1
433,purchase,computers.notebook,hp,285.51,521162064,5622db13-66d1-4167-8bfa-b4f184aed29a,computers,notebook,1


In [7]:
com_once.drop(columns = ['event_type', 'user_session', 'main_category', 'Frequency'], inplace=True)
com_once = com_once.reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
com_once_user.drop(columns = ['event_type', 'main_category', 'subcategory','Frequency'], inplace=True)
com_once_user = com_once_user.reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
com_once

,category_code,brand,price,user_id,subcategory
0,computers.peripherals.printer,epson,116.73,513489079,printer
1,computers.components.hdd,kingston,66.67,523117953,hdd
2,computers.notebook,acer,532.81,549790258,notebook
3,computers.notebook,hp,285.51,555467053,notebook
4,computers.notebook,hp,285.51,521162064,notebook
...,...,...,...,...,...
9348,computers.notebook,lenovo,365.26,518406442,notebook
9349,computers.desktop,hp,720.48,566258164,desktop
9350,computers.peripherals.mouse,x-game,5.03,566267483,mouse
9351,computers.notebook,asus,352.96,513741972,notebook


In [13]:
com_once_user.head()

,category_code,brand,price,user_id,user_session
0,computers.peripherals.printer,epson,116.73,513489079,63de1120-9d48-45d0-b024-500d1b95d71f
1,computers.components.hdd,kingston,66.67,523117953,629a0485-36c9-4a80-9f02-4a96382c3059
2,computers.notebook,acer,532.81,549790258,56272bb8-9f35-4412-9f15-624c8ecda0f1
3,computers.notebook,hp,285.51,555467053,8d2582ff-8031-4b06-819b-a653eb1cb832
4,computers.notebook,hp,285.51,521162064,5622db13-66d1-4167-8bfa-b4f184aed29a


In [14]:
com_once_user.isna().sum()

category_code      0
brand            228
price              0
user_id            0
user_session       0
dtype: int64

In [15]:
com_once.isna().sum()

category_code      0
brand            228
price              0
user_id            0
subcategory        0
dtype: int64

In [9]:
com_once_user.dropna(inplace=True)

In [10]:
com_once.dropna(inplace=True)

### computers 제품간 추천

In [11]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_tfidf = tfidf.fit_transform(com_once['category_code'])

In [153]:
dtm_tfidf_df= pd.DataFrame(dtm_tfidf.todense(), columns=tfidf.get_feature_names_out())
dtm_tfidf_df

,camera,components,computers,cooler,cpu,desktop,ebooks,hdd,keyboard,memory,monitor,motherboard,mouse,notebook,peripherals,power_supply,printer,videocards
0,0.0,0.00000,0.237006,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.613976,0.0,0.752903,0.0
1,0.0,0.59821,0.184131,0.0,0.0,0.000000,0.0,0.779898,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
2,0.0,0.00000,0.534484,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.845179,0.000000,0.0,0.000000,0.0
3,0.0,0.00000,0.534484,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.845179,0.000000,0.0,0.000000,0.0
4,0.0,0.00000,0.534484,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.845179,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,0.0,0.00000,0.534484,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.845179,0.000000,0.0,0.000000,0.0
9121,0.0,0.00000,0.301918,0.0,0.0,0.953334,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
9122,0.0,0.00000,0.191205,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.847403,0.000000,0.495327,0.0,0.000000,0.0
9123,0.0,0.00000,0.534484,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.845179,0.000000,0.0,0.000000,0.0


In [12]:
cosine_matrix = cosine_similarity(dtm_tfidf, dtm_tfidf)

In [13]:
np.round(cosine_matrix, 4)

array([[1.    , 0.0436, 0.1267, ..., 0.3494, 0.1267, 0.1267],
       [0.0436, 1.    , 0.0984, ..., 0.0352, 0.0984, 0.0984],
       [0.1267, 0.0984, 1.    , ..., 0.1022, 1.    , 1.    ],
       ...,
       [0.3494, 0.0352, 0.1022, ..., 1.    , 0.1022, 0.1022],
       [0.1267, 0.0984, 1.    , ..., 0.1022, 1.    , 1.    ],
       [0.1267, 0.0984, 1.    , ..., 0.1022, 1.    , 1.    ]])

In [22]:
category = {}
for i, c in enumerate(com_once['user_id']): 
  category[i] = c

category_id = {}
for i, c in category.items():
  category_id[c] = i

In [23]:
category_id

{513489079: 0,
 523117953: 1,
 549790258: 2,
 555467053: 3,
 521162064: 4,
 555468492: 5,
 538858285: 6,
 530820988: 7,
 533828408: 8,
 521819296: 9,
 512742880: 11,
 513250707: 12,
 512604234: 13,
 513861839: 14,
 531542244: 15,
 550000136: 16,
 550982997: 17,
 515567849: 18,
 515856194: 19,
 546884787: 20,
 546202787: 21,
 555485978: 22,
 554837844: 23,
 555485235: 24,
 520552458: 25,
 515104074: 26,
 517460166: 27,
 528769567: 28,
 517465372: 29,
 514531448: 30,
 526799174: 31,
 517529110: 32,
 536387441: 33,
 512374444: 34,
 514953614: 35,
 555495717: 36,
 555491795: 37,
 530123083: 38,
 539257365: 39,
 530233653: 40,
 544411220: 41,
 536838713: 42,
 555498267: 43,
 525197121: 44,
 553088582: 45,
 517010638: 46,
 512953546: 47,
 527790692: 48,
 551008846: 49,
 541539898: 50,
 516205307: 51,
 518105307: 52,
 512644482: 53,
 523099429: 54,
 541270844: 55,
 554633294: 57,
 555513645: 58,
 543745179: 59,
 522366714: 60,
 515582362: 61,
 544325552: 62,
 553629184: 63,
 522003699: 65,
 5

In [29]:
idx = category_id[523117953]
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(29, 1.0), (46, 1.0), (67, 1.0), (84, 1.0), (92, 1.0)]

In [30]:
com_once.loc[com_once['user_id'] == 523117953]

,category_code,brand,price,user_id,subcategory
1,computers.components.hdd,kingston,66.67,523117953,hdd


In [35]:
com_once.iloc[[29]]

,category_code,brand,price,user_id,subcategory
29,computers.components.hdd,apacer,24.2,517465372,hdd


In [34]:
com_once.iloc[[46]]

,category_code,brand,price,user_id,subcategory
48,computers.components.hdd,kingston,38.32,517010638,hdd


In [33]:
com_once.iloc[[67]]

,category_code,brand,price,user_id,subcategory
71,computers.components.hdd,toshiba,39.02,531895455,hdd


In [32]:
com_once.iloc[[84]]

,category_code,brand,price,user_id,subcategory
88,computers.components.hdd,apacer,34.75,518667476,hdd


In [31]:
com_once.iloc[[92]]

,category_code,brand,price,user_id,subcategory
96,computers.components.hdd,seagate,66.8,550318201,hdd


### computers 유저간 추천

In [36]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_tfidf_user = tfidf.fit_transform(com_once_user['user_session'])

In [136]:
dtm_tfidf_user_df= pd.DataFrame(dtm_tfidf_user.todense(), columns=tfidf.get_feature_names_out())
dtm_tfidf_user_df

,0002,00025dab,0002c5ea,0005,0006bf97f732,0007ab79ced4,0011,0012d784,001312a0142e,00153b60a6e5,...,ffeb,ffed,ffee,fff6,fff60634,fff661e40162,fff9,fffa,fffac901e3c9,fffc
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
cosine_matrix = cosine_similarity(dtm_tfidf_user, dtm_tfidf_user)

In [38]:
np.round(cosine_matrix, 4)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [39]:
user = {}
for i, c in enumerate(com_once_user['user_session']): 
  user[i] = c

user_id = {}
for i, c in user.items():
  user_id[c] = i

In [40]:
user_id

{'63de1120-9d48-45d0-b024-500d1b95d71f': 0,
 '629a0485-36c9-4a80-9f02-4a96382c3059': 1,
 '56272bb8-9f35-4412-9f15-624c8ecda0f1': 2,
 '8d2582ff-8031-4b06-819b-a653eb1cb832': 3,
 '5622db13-66d1-4167-8bfa-b4f184aed29a': 4,
 '739c3ef2-012f-41b7-8450-01ca3866d6ff': 5,
 '0f65dee0-ae4d-460e-bb66-3da1bbbaec6b': 6,
 '9b9cc736-3daf-4451-a2d5-05c53011c2e5': 7,
 '5cfee4a3-d064-471e-a580-503dae0b490e': 8,
 '3a8a2e45-3c9b-4d1a-868f-9fb732f91bab': 9,
 '933010d0-2c90-4adc-a161-ec953aa99b91': 11,
 'b610c284-5924-4211-932a-340f98c1c700': 12,
 '52fad75b-1e60-4dec-bc22-a51d3d5680ae': 13,
 '5a14310e-3c18-4383-b584-7f56aefcd535': 14,
 '970f59a8-9af7-4fa0-bbbb-d81e7146ab9f': 15,
 '01d9b508-1e1d-4ca8-8403-d7962eaf34b6': 16,
 'e874dc74-0630-4b3a-b049-9dbf53d745c9': 17,
 '1fcdca81-f119-4556-b044-0d7a24fb7b61': 18,
 '7eb1c0d9-4b45-4682-bf26-c650b6d8c8fe': 19,
 'a31d70d9-ea3b-4b04-b247-9368618f8b2b': 20,
 '5ac10175-3214-4b5f-8906-2db4ef9bedcb': 21,
 'cb6a8583-b035-4d76-8583-d3a8ee969020': 22,
 '7e5ba25e-98ef-42ff

In [41]:
idx_category = user_id['629a0485-36c9-4a80-9f02-4a96382c3059']
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx_category]) if i != idx_category] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 10개의 인덱스와 유사도를 추출 

[(3212, 0.191554480946969),
 (536, 0.17333571871089953),
 (377, 0.17227939898059863),
 (4214, 0.17227939898059863),
 (0, 0.0)]

In [42]:
com_once_user.loc[com_once_user['user_session'] == '629a0485-36c9-4a80-9f02-4a96382c3059']

,category_code,brand,price,user_id,user_session
1,computers.components.hdd,kingston,66.67,523117953,629a0485-36c9-4a80-9f02-4a96382c3059


In [43]:
com_once_user.iloc[[3312]]

,category_code,brand,price,user_id,user_session
3387,computers.notebook,asus,424.7,521542628,30e9ecd1-8a68-4cb3-88cb-debff0f17a3d


In [46]:
com_once_user.iloc[[536]]

,category_code,brand,price,user_id,user_session
549,computers.notebook,hp,320.35,551478118,9c664cb4-9311-4a80-bd17-fca008e49c98


In [47]:
com_once_user.iloc[[377]]

,category_code,brand,price,user_id,user_session
388,computers.components.hdd,transcend,45.05,548837228,165a2320-9850-4a80-845b-1b1fcbc1143d


In [48]:
com_once_user.iloc[[4214]]

,category_code,brand,price,user_id,user_session
4309,computers.notebook,asus,540.53,559683462,70d3de0c-ce16-4a80-b6b0-e69b3393af9a


### appliances 제품간 추천

In [5]:
appli_once = RFM_without_view.loc[(RFM_without_view['Frequency'] < 2) & (RFM_without_view['event_type'] == 'purchase') & (RFM_without_view['main_category'] == 'appliances')]

In [6]:
appli_once.drop(columns=['event_type','user_id','main_category','Frequency', 'RFM_label'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
appli_once = appli_once.reset_index(drop=True)

In [8]:
appli_once.head()

,category_code,brand,price,user_session,subcategory
0,appliances.environment.air_heater,oasis,28.03,b67cdbcb-b073-4271-b365-803c6fce53b0,air_heater
1,appliances.kitchen.meat_grinder,philips,120.95,94b897fc-80c6-492c-a456-707398c7cc8e,meat_grinder
2,appliances.kitchen.washer,lg,308.65,5e227e58-6117-419c-b45d-ac54be44716d,washer
3,appliances.kitchen.refrigerators,NaN,204.33,788ec8fe-f198-4484-b911-ba5841c55c68,refrigerators
4,appliances.personal.hair_cutter,moser,47.88,35323009-6410-4412-930e-e7855384a5da,hair_cutter


In [9]:
appli_once.shape

(31936, 5)

In [50]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_appli = tfidf.fit_transform(appli_once['category_code'])

In [33]:
dtm_appli_df= pd.DataFrame(dtm_appli.todense(), columns=tfidf.get_feature_names_out())
dtm_appli_df

,air_conditioner,air_heater,appliances,blender,coffee_grinder,coffee_machine,dishwasher,environment,fan,grill,...,oven,personal,refrigerators,scales,sewing_machine,steam_cooker,toster,vacuum,washer,water_heater
0,0.0,0.862929,0.196043,0.0,0.0,0.0,0.0,0.465747,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
1,0.0,0.000000,0.210327,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
2,0.0,0.000000,0.317425,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.82639,0.0
3,0.0,0.000000,0.293136,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.85416,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
4,0.0,0.000000,0.145957,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.631627,0.00000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31931,0.0,0.000000,0.185239,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
31932,0.0,0.000000,0.317425,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.82639,0.0
31933,0.0,0.000000,0.226555,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
31934,0.0,0.000000,0.212498,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.977161,0.0,0.0,0.0,0.00000,0.0


In [51]:
cosine_matrix = cosine_similarity(dtm_appli, dtm_appli)

In [52]:
np.round(cosine_matrix, 4)

array([[1.    , 0.0412, 0.0622, ..., 0.0444, 0.0417, 0.0622],
       [0.0412, 1.    , 0.2101, ..., 0.15  , 0.0447, 0.2101],
       [0.0622, 0.2101, 1.    , ..., 0.2263, 0.0675, 1.    ],
       ...,
       [0.0444, 0.15  , 0.2263, ..., 1.    , 0.0481, 0.2263],
       [0.0417, 0.0447, 0.0675, ..., 0.0481, 1.    , 0.0675],
       [0.0622, 0.2101, 1.    , ..., 0.2263, 0.0675, 1.    ]])

In [64]:
category = {}
for i, c in enumerate(appli_once['subcategory']): 
  category[i] = c

category_id = {}
for i, c in category.items():
  category_id[c] = i

In [74]:
idx = category_id['air_heater']
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(0, 1.0), (30, 1.0), (47, 1.0), (58, 1.0), (63, 1.0)]

In [80]:
appli_once.iloc[[967]]

,category_code,brand,price,user_session,subcategory
967,appliances.environment.air_heater,tefal,167.29,f9b77efc-e1f4-44f7-a30c-de1176c6689c,air_heater


In [81]:
appli_once.iloc[[0]]

,category_code,brand,price,user_session,subcategory
0,appliances.environment.air_heater,oasis,28.03,b67cdbcb-b073-4271-b365-803c6fce53b0,air_heater


In [82]:
appli_once.iloc[[47]]

,category_code,brand,price,user_session,subcategory
47,appliances.environment.air_heater,NaN,10.17,4f885f6f-df02-40a1-b640-bd6c44861709,air_heater


In [83]:
appli_once.iloc[[58]]

,category_code,brand,price,user_session,subcategory
58,appliances.environment.air_heater,midea,47.62,f7e2c109-b4d3-4614-bdc3-3afcdd3f4937,air_heater


In [84]:
appli_once.iloc[[63]]

,category_code,brand,price,user_session,subcategory
63,appliances.environment.air_heater,elenberg,33.44,cd592886-cf59-4d47-81a3-3a26f1605306,air_heater


### appliances 유저간 추천

In [9]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_appli_user = tfidf.fit_transform(appli_once['user_session'])

In [ ]:
dtm_appli_user_df= pd.DataFrame(dtm_appli_user.todense(), columns=tfidf.get_feature_names_out())
dtm_appli_user_df

In [10]:
cosine_matrix = cosine_similarity(dtm_appli_user, dtm_appli_user)

In [11]:
np.round(cosine_matrix, 4)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [12]:
user = {}
for i, c in enumerate(appli_once['user_session']): 
  user[i] = c

id = {}
for i, c in user.items():
  id[c] = i

In [13]:
id

{'b67cdbcb-b073-4271-b365-803c6fce53b0': 0,
 '94b897fc-80c6-492c-a456-707398c7cc8e': 1,
 '5e227e58-6117-419c-b45d-ac54be44716d': 2,
 '788ec8fe-f198-4484-b911-ba5841c55c68': 3,
 '35323009-6410-4412-930e-e7855384a5da': 4,
 'd244ccce-4dbb-4ae6-af48-98eb36069e19': 5,
 '87f507c9-8679-4b25-87f0-dc90e36b7b6d': 6,
 '09da2c0e-55cf-4017-99eb-2fb64f550404': 7,
 '6f7b28d9-676c-4840-945a-184bbf955181': 8,
 '0b58cee9-ab6f-452a-8b17-e2239707e299': 9,
 '87515b74-457c-4272-8fcf-5739f0196b4e': 10,
 '62c80e92-1f6c-43c2-8e28-87a3cf7a9243': 11,
 'ddebe7b1-5376-49e9-8e87-14b575a30174': 12,
 'd36a86cc-b609-4b78-8040-389d39fd35b2': 13,
 'c62afcb5-b721-4a46-88ca-2c7f9ee42f07': 14,
 '9bcee4ed-01fa-468d-906b-565bc222fa21': 15,
 'dc03489d-7086-4282-b4d0-bd7aee07d491': 16,
 'a3466904-e855-40c5-b3bd-6739e29ea603': 17,
 'ae16d5ad-53d5-4f4b-9d79-4d32ab99641e': 18,
 'c7b658a0-ffec-4e5a-9c39-05193809de53': 19,
 '52a857d6-b4db-42cb-bf03-b0b66cd36c72': 20,
 'df26d914-fe85-440b-86be-15d6de9115a6': 21,
 '7f24153b-a9ad-4274

In [14]:
idx = id['d873346c-be5c-4208-8663-d66006d8983b']
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(19530, 0.19333999230186524),
 (19531, 0.19333999230186524),
 (12960, 0.1889467210201055),
 (3196, 0.18810067021720445),
 (25904, 0.18759032378720253)]

In [15]:
appli_once.iloc[[1073]]

,category_code,brand,price,user_session,subcategory
1073,appliances.kitchen.refrigerators,NaN,210.07,d873346c-be5c-4208-8663-d66006d8983b,refrigerators


In [16]:
appli_once.iloc[[19530]]

,category_code,brand,price,user_session,subcategory
19530,appliances.environment.vacuum,samsung,69.47,37e5d81c-3adc-4a38-8663-0115c212dca3,vacuum


In [17]:
appli_once.iloc[[19531]]

,category_code,brand,price,user_session,subcategory
19531,appliances.environment.vacuum,samsung,69.47,37e5d81c-3adc-4a38-8663-0115c212dca3,vacuum


In [18]:
appli_once.iloc[[12960]]

,category_code,brand,price,user_session,subcategory
12960,appliances.kitchen.kettle,redmond,28.01,64306efe-1112-40cd-be5c-d0b08030d4e0,kettle


In [19]:
appli_once.iloc[[3196]]

,category_code,brand,price,user_session,subcategory
3196,appliances.environment.air_heater,NaN,7.31,b90430b3-db02-4303-be5c-d1fb65207e9f,air_heater


In [20]:
appli_once.iloc[[25904]]

,category_code,brand,price,user_session,subcategory
25904,appliances.environment.vacuum,samsung,218.77,322b1c44-20bf-438b-8663-79219fbc43f6,vacuum


### electronics 제품간 추천
- segment2에 대한 추천

In [31]:
elec_once = RFM_without_view.loc[(RFM_without_view['Frequency'] < 2) & (RFM_without_view['event_type'] == 'purchase') & (RFM_without_view['main_category'] == 'electronics') & (RFM_without_view['RFM_label'] == 'segment2')]

In [32]:
elec_once.drop(columns=['event_type', 'main_category','Frequency','RFM_label'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [35]:
elec_once.drop(columns=['user_id','user_session', 'subcategory'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [37]:
elec_once.reset_index(drop=True, inplace=True)

In [38]:
elec_once.head()

,product_id,category_code,brand,price
0,1005104,electronics.smartphone,apple,1007.74
1,1005105,electronics.smartphone,apple,1397.09
2,1004258,electronics.smartphone,apple,731.47
3,1005129,electronics.smartphone,apple,1397.72
4,1003310,electronics.smartphone,apple,696.03


In [39]:
elec_once.shape

(7114, 4)

In [40]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_elec = tfidf.fit_transform(elec_once['category_code'])

In [41]:
cosine_matrix = cosine_similarity(dtm_elec, dtm_elec)

In [42]:
np.round(cosine_matrix, 4)

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [44]:
category = {}
for i, c in enumerate(elec_once['product_id']): 
  category[i] = c

category_id = {}
for i, c in category.items():
  category_id[c] = i

In [45]:
category_id

{1002101: 6177,
 1002524: 7019,
 1002525: 7049,
 1002528: 6808,
 1002531: 7006,
 1002532: 6978,
 1002535: 6317,
 1002536: 4509,
 1002540: 6915,
 1002544: 6941,
 1002545: 3213,
 1002547: 6015,
 1002548: 2763,
 1002628: 2857,
 1002629: 6628,
 1002633: 6824,
 1002634: 6333,
 1002796: 6740,
 1003028: 4818,
 1003153: 6184,
 1003281: 1459,
 1003304: 6181,
 1003305: 5684,
 1003306: 6987,
 1003307: 6072,
 1003308: 5185,
 1003309: 7080,
 1003310: 6911,
 1003311: 7061,
 1003312: 6875,
 1003313: 6993,
 1003314: 6847,
 1003315: 5109,
 1003316: 7041,
 1003317: 7087,
 1003318: 7066,
 1003319: 6600,
 1003363: 3772,
 1003523: 980,
 1003533: 4822,
 1003549: 4838,
 1003712: 6207,
 1003713: 4762,
 1003714: 946,
 1003751: 4028,
 1003768: 101,
 1003800: 6085,
 1003801: 4569,
 1003879: 4125,
 1003954: 5760,
 1003965: 2959,
 1003973: 4547,
 1004016: 2842,
 1004133: 1478,
 1004158: 6940,
 1004209: 5356,
 1004225: 7096,
 1004226: 6972,
 1004227: 7020,
 1004228: 6073,
 1004229: 6944,
 1004230: 6068,
 1004231: 1

In [47]:
idx = category_id[21410400]
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(7, 1.0000000000000002),
 (31, 1.0000000000000002),
 (72, 1.0000000000000002),
 (96, 1.0000000000000002),
 (112, 1.0000000000000002)]

In [48]:
elec_once.iloc[[7068]]

,product_id,category_code,brand,price
7068,21410400,electronics.clocks,tissot,679.55


In [49]:
elec_once.iloc[[7]]

,product_id,category_code,brand,price
7,5100797,electronics.clocks,garmin,2393.88


In [50]:
elec_once.iloc[[31]]

,product_id,category_code,brand,price
31,5100856,electronics.clocks,apple,616.13


In [51]:
elec_once.iloc[[72]]

,product_id,category_code,brand,price
72,5100899,electronics.clocks,apple,643.26


In [52]:
elec_once.iloc[[96]]

,product_id,category_code,brand,price
96,5100899,electronics.clocks,apple,643.26


In [53]:
elec_once.iloc[[112]]

,product_id,category_code,brand,price
112,5100884,electronics.clocks,garmin,1248.66


### electronics 유저간 추천
- segment2 에 대한 추천

In [60]:
elec_user = RFM_without_view.loc[(RFM_without_view['Frequency'] < 2) & (RFM_without_view['event_type'] == 'purchase') & (RFM_without_view['main_category'] == 'electronics') & (RFM_without_view['RFM_label'] == 'segment2')]

In [61]:
elec_user.drop(columns=['product_id','event_type', 'main_category', 'subcategory', 'Frequency', 'RFM_label'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [62]:
elec_user.reset_index(drop=True, inplace=True)

In [63]:
elec_user.head()

,category_code,brand,price,user_id,user_session
0,electronics.smartphone,apple,1007.74,553492314,859d6cd6-a3d0-4049-8a0e-212a3d864d48
1,electronics.smartphone,apple,1397.09,551646372,bbef1e78-8279-41ac-8bd7-e459e1ad9f3e
2,electronics.smartphone,apple,731.47,553492314,859d6cd6-a3d0-4049-8a0e-212a3d864d48
3,electronics.smartphone,apple,1397.72,551646372,bbef1e78-8279-41ac-8bd7-e459e1ad9f3e
4,electronics.smartphone,apple,696.03,563963172,cf5993a5-17c9-41ef-a9c4-d1bfc624402a


In [64]:
elec_user.shape

(7114, 5)

In [65]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_elec_user = tfidf.fit_transform(elec_user['user_session'])

In [66]:
cosine_matrix = cosine_similarity(dtm_elec_user, dtm_elec_user)

In [67]:
np.round(cosine_matrix, 4)

array([[1., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [68]:
user = {}
for i, c in enumerate(elec_user['user_id']): 
  user[i] = c

user_id = {}
for i, c in user.items():
  user_id[c] = i

In [69]:
user_id

{553492314: 2,
 551646372: 9,
 563963172: 4,
 547101682: 5,
 527225862: 6,
 563963947: 7,
 515414031: 8,
 534249929: 10,
 546373721: 11,
 556763735: 12,
 560361981: 13,
 514988855: 14,
 513465845: 15,
 516323877: 16,
 540817817: 17,
 521712163: 18,
 547308910: 19,
 562715444: 20,
 556310310: 21,
 563968655: 36,
 561582483: 27,
 563401481: 24,
 563970418: 25,
 563969332: 26,
 561962546: 28,
 563970672: 29,
 521203750: 30,
 542185768: 31,
 521843962: 32,
 540741354: 37,
 551624904: 35,
 553791455: 38,
 560436426: 39,
 552061422: 40,
 522554425: 41,
 563973738: 42,
 526750679: 43,
 542778634: 44,
 548144606: 76,
 519043940: 50,
 551322808: 47,
 521397658: 48,
 536589557: 49,
 517666246: 51,
 516251196: 52,
 562716401: 53,
 563976477: 54,
 513206475: 55,
 563979640: 56,
 563977910: 63,
 552311423: 58,
 551076980: 60,
 513286962: 61,
 517686800: 62,
 519877754: 64,
 536123941: 65,
 563981537: 66,
 563968953: 67,
 547787231: 70,
 520196501: 69,
 563978746: 71,
 559025270: 72,
 538518045: 73,

In [71]:
idx = user_id[565966895]
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(7067, 1.0000000000000002),
 (7068, 1.0000000000000002),
 (4306, 0.18505806770304392),
 (2757, 0.17569227655160585),
 (1367, 0.17409519720961947)]

In [73]:
elec_user.loc[elec_user['user_id'] == 565966895]

,category_code,brand,price,user_id,user_session
7067,electronics.clocks,tissot,1050.22,565966895,b3a7e0f4-26a4-42e1-af6c-6ad9ab87636c
7068,electronics.clocks,tissot,679.55,565966895,b3a7e0f4-26a4-42e1-af6c-6ad9ab87636c
7071,electronics.clocks,tissot,741.33,565966895,b3a7e0f4-26a4-42e1-af6c-6ad9ab87636c


In [74]:
elec_user.iloc[[4306]]

,category_code,brand,price,user_id,user_session
4306,electronics.smartphone,samsung,771.94,544053861,20da16b2-26a4-46d8-b608-fb35c0d8575e


In [75]:
elec_user.iloc[[2757]]

,category_code,brand,price,user_id,user_session
2757,electronics.smartphone,apple,1692.81,564816718,f2f95827-42e1-431e-9172-63d10b4e6648


In [76]:
elec_user.iloc[[1367]]

,category_code,brand,price,user_id,user_session
1367,electronics.smartphone,apple,1043.97,512951857,ce1e3b01-1a34-42e1-9eff-4c76a72c7cac
